# **Pre-Processing**




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import time

In [ ]:
dataset=pd.read_csv("/content/drive/MyDrive/Projects/Sentiment Analysis Using Reviews/Coding /Colab Directory/cell_phone_and_accessories_review.csv")
dataset.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,0,5.0,True,"08 4, 2014",A24E3SXTC62LJI,7508492919,{'Color:': ' Bling'},Claudia Valdivia,Looks even better in person. Be careful to not...,Can't stop won't stop looking at it,1407110400,NaN,NaN
1,1,5.0,True,"02 12, 2014",A269FLZCB4GIPV,7508492919,NaN,sarah ponce,When you don't want to spend a whole lot of ca...,1,1392163200,NaN,NaN
2,2,3.0,True,"02 8, 2014",AB6CHQWHZW4TV,7508492919,NaN,Kai,"so the case came on time, i love the design. I...",Its okay,1391817600,NaN,NaN
3,3,2.0,True,"02 4, 2014",A1M117A53LEI8,7508492919,NaN,Sharon Williams,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY...,CASE,1391472000,NaN,NaN
4,4,4.0,True,"02 3, 2014",A272DUT8M88ZS8,7508492919,NaN,Bella Rodriguez,"I liked it because it was cute, but the studs ...",Cute!,1391385600,NaN,NaN


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128437 entries, 0 to 1128436
Data columns (total 13 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   Unnamed: 0      1128437 non-null  int64  
 1   overall         1128437 non-null  float64
 2   verified        1128437 non-null  bool   
 3   reviewTime      1128437 non-null  object 
 4   reviewerID      1128437 non-null  object 
 5   asin            1128437 non-null  object 
 6   style           605241 non-null   object 
 7   reviewerName    1128281 non-null  object 
 8   reviewText      1127669 non-null  object 
 9   summary         1127912 non-null  object 
 10  unixReviewTime  1128437 non-null  int64  
 11  vote            92034 non-null    object 
 12  image           27107 non-null    object 
dtypes: bool(1), float64(1), int64(2), object(9)
memory usage: 104.4+ MB


In [ ]:
clean1=dataset.drop(["Unnamed: 0","verified","reviewTime","style","reviewerName","summary","unixReviewTime","vote","image"], axis=1)
clean1.dropna(how='any',inplace=True)
clean1

,overall,reviewerID,asin,reviewText
0,5.0,A24E3SXTC62LJI,7508492919,Looks even better in person. Be careful to not...
1,5.0,A269FLZCB4GIPV,7508492919,When you don't want to spend a whole lot of ca...
2,3.0,AB6CHQWHZW4TV,7508492919,"so the case came on time, i love the design. I..."
3,2.0,A1M117A53LEI8,7508492919,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY...
4,4.0,A272DUT8M88ZS8,7508492919,"I liked it because it was cute, but the studs ..."
...,...,...,...,...
1128432,4.0,A1QWMCG1FNEP3A,B01HJC7N4C,Good for viewing. But doesn't have a button or...
1128433,5.0,A3FOBEJ9UVUTR3,B01HJC7N4C,I was given the Rockrok 3D VR Glasses Headset ...
1128434,5.0,AMUEAMKB4E33M,B01HJC7N4C,Super Fun! The RockRoc 3d vr headset is waaaay...
1128435,5.0,A2EV91MMOJ3IL4,B01HJC7N4C,Love it!\n\nI've had other VR glasses which al...


In [ ]:
clean1["overall"].value_counts()

5.0    706432
4.0    184351
3.0     98214
1.0     81506
2.0     57166
Name: overall, dtype: int64

In [ ]:
def partition(x):
    if x<3:
        return -1
    if x==3:
        return 0
    return 1

clean1['overall']=clean1['overall'].map(partition)

In [ ]:
clean1=clean1.drop_duplicates()
clean1=clean1.reset_index(drop=True)
clean1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1123322 entries, 0 to 1123321
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   overall     1123322 non-null  int64 
 1   reviewerID  1123322 non-null  object
 2   asin        1123322 non-null  object
 3   reviewText  1123322 non-null  object
dtypes: int64(1), object(3)
memory usage: 34.3+ MB


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english')) #set of stopwords
print(stop_words)

{'below', 'because', 'it', 's', "doesn't", 'before', 'me', "weren't", 'why', "couldn't", 'that', 'over', 'so', 'hadn', "hadn't", 'yourselves', 'with', 'further', "shouldn't", 'i', 'you', 'your', 'them', 'same', 'they', "mightn't", 'was', 'be', 'other', "wouldn't", 'y', 'some', "it's", 'her', "you've", 'o', 'isn', 'about', 'ourselves', 'having', 'each', "won't", 'an', 'too', 'theirs', 'won', 'which', 'here', 'hers', "didn't", 'mustn', 'these', 't', 'where', 'and', 'again', 'of', 'any', 'few', 'at', 'hasn', 'haven', 'their', "needn't", 'will', 'its', 'as', 'doesn', 'were', 'didn', 'until', 'himself', 'who', 'herself', 'can', 'how', 'nor', 'he', 'most', 'yours', "shan't", "you'd", 'is', 'those', 'are', 'against', 've', 'ours', 'been', 'needn', "haven't", 'the', 'ma', 'but', 'through', 'once', "aren't", 'itself', 'd', 'm', 're', 'did', 'being', 'to', 'now', 'yourself', "don't", 'don', 'wouldn', 'my', 'then', 'than', 'shan', "wasn't", 'by', 'very', 'there', 'own', 'after', 'doing', 'she', '

In [ ]:
import nltk
sno = nltk.stem.SnowballStemmer('english') #initialising the snowball stemmer

# **Pre-Processing Function**

In [ ]:
import re
def data_cleaning(series):
    i = 0
    string = ""
    final_string = []   
    for sent in series.values:
        filtered_sent = []
        sent = re.sub('[^a-zA-Z0-9\n]', ' ', sent) #special characters
        sent = re.sub('\s+',' ', sent) #multiple spaces
        sent = sent.lower() #lowercasing
        for word in sent.split():
            if word not in stop_words and len(word)>2:
                word = sno.stem(word) #Stemming
                filtered_sent.append(word)
        string = " ".join(filtered_sent) # Cleaned sentence
        final_string.append(string) # List of cleaned sentences
        i+=1
    return final_string

In [ ]:
start = time.time()
final_string= data_cleaning(clean1['reviewText'])
end = time.time()
print("Total time in minutes = ", (end-start)/60)

Total time in minutes =  7.25913716952006


In [ ]:
clean1['CleanedText']=final_string
clean1.head()

,overall,reviewerID,asin,reviewText,CleanedText
0,1,A24E3SXTC62LJI,7508492919,Looks even better in person. Be careful to not...,look even better person care drop phone often ...
1,1,A269FLZCB4GIPV,7508492919,When you don't want to spend a whole lot of ca...,want spend whole lot cash want great deal shop...
2,0,AB6CHQWHZW4TV,7508492919,"so the case came on time, i love the design. I...",case came time love design actual miss stud no...
3,-1,A1M117A53LEI8,7508492919,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY...,care gave gift okay expect
4,1,A272DUT8M88ZS8,7508492919,"I liked it because it was cute, but the studs ...",like cute stud fall easili protect phone would...


In [ ]:
clean1['CleanedText'].dropna(inplace=True)

In [ ]:
data=clean1[['overall','CleanedText']].copy()
del clean1
data.head()

,overall,CleanedText
0,1,look even better person care drop phone often ...
1,1,want spend whole lot cash want great deal shop...
2,0,case came time love design actual miss stud no...
3,-1,care gave gift okay expect
4,1,like cute stud fall easili protect phone would...


# **POS Tagging**

In [ ]:
from nltk.corpus import wordnet as wn
from nltk import pos_tag
import nltk
def data_pos(series):
    i = 0
    string = ""
    final_string = []
    for sentence in series.values:
        filtered_sent = []
        sentence = nltk.word_tokenize(sentence)
        sent = pos_tag(sentence)
        for (x,y) in sent:
            if y in ('JJ') or y in ('RB'):
                filtered_sent.append(x)
        string = " ".join(filtered_sent) # Cleaned sentence
        final_string.append(string) # List of cleaned sentences
        i+=1
    return final_string

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
start = time.time()
final_string1= data_pos(data['CleanedText'])
end = time.time()
print("Total time in minutes = ", (end-start)/60)

Total time in minutes =  32.13815416495005


In [ ]:
data['CleanedText1']=final_string1
data.head()

,overall,CleanedText,CleanedText1
0,1,look even better person care drop phone often ...,care fit perfect overal
1,1,want spend whole lot cash want great deal shop...,want spend whole great
2,0,case came time love design actual miss stud no...,actual notic sloppi bow notic notic
3,-1,care gave gift okay expect,
4,1,like cute stud fall easili protect phone would...,cute recommend buy


In [ ]:
data_final=data[['overall','CleanedText1']].copy()
del data
data_final.head()

,overall,CleanedText1
0,1,care fit perfect overal
1,1,want spend whole great
2,0,actual notic sloppi bow notic notic
3,-1,
4,1,cute recommend buy


In [ ]:
data_final['CleanedText1'].dropna(inplace=True)

# **Getting Data Model Ready**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score

In [ ]:
data_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1123322 entries, 0 to 1123321
Data columns (total 2 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   overall       1123322 non-null  int64 
 1   CleanedText1  1123322 non-null  object
dtypes: int64(1), object(1)
memory usage: 17.1+ MB


In [ ]:
data_final["overall"].value_counts()

 1    887475
-1    138001
 0     97846
Name: overall, dtype: int64

In [ ]:
data_final=data_final[:500000]
data_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   overall       500000 non-null  int64 
 1   CleanedText1  500000 non-null  object
dtypes: int64(1), object(1)
memory usage: 7.6+ MB


In [ ]:
X=data_final['CleanedText1']
y=data_final['overall']

In [ ]:
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size = 0.25, shuffle = False)
X_train_raw.shape, y_train.shape, X_test_raw.shape, y_test.shape

((375000,), (375000,), (125000,), (125000,))

In [ ]:
bow = CountVectorizer()
X_train = bow.fit_transform(X_train_raw.values.astype('str'))

In [ ]:
X_train.shape

(375000, 20882)

In [ ]:
X_test = bow.transform(X_test_raw.values.astype('str'))
X_test.shape

(125000, 20882)

# **Implementing Model**

In [ ]:
start = time.time()
classifier = MultinomialNB(alpha=1.0)
classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_test)
scores = cross_val_score(classifier, X, y, cv=5)
print(scores)
end = time.time()
print("Total time in minutes = ", (end-start)/60)

[nan nan nan nan nan]
Total time in minutes =  0.007774738470713297


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: 'real'

  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: 'care fit perfect overal'

  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: 'care fit perfect overal'

  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarnin

In [ ]:
cm = confusion_matrix(y_test,y_pred)
accuracy = float(cm.diagonal().sum())/len(y_test)
print("\nAccuracy Of Naive Bayes For The Given Dataset : ", accuracy)


Accuracy Of Naive Bayes For The Given Dataset :  0.813272


In [ ]:
confusion_matrix(y_test, y_pred)

array([[ 2612,   280, 11230],
       [  660,   351,  9142],
       [ 1355,   674, 98696]])

In [ ]:
from sklearn.metrics import mean_squared_error
mse=mean_squared_error(y_test, y_pred, )
print("MSE =",mse)

MSE = 0.488768
